In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.losses import binary_crossentropy
from keras.optimizers import Adam
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import keras
import json

Using TensorFlow backend.


In [8]:
model = keras.models.load_model('model-conv1d.h5')

In [16]:
with open("categories.json","r") as fp:
    categories = json.load(fp)

encoder = LabelEncoder()
encoder.fit(categories)

with open("LabeledEvent.json","r") as fp:
    train = json.load(fp)
x = [event['description']+" "+event['title'] for event in train if 'description' in event and 'title' in event]
y = [event['primaryCategory'] for event in train if 'primaryCategory' in event]

In [17]:
tokenizer = Tokenizer(num_words=5000, lower=True)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)
x = pad_sequences(sequences, maxlen=180, padding="post")

In [12]:
predictedLabel = model.predict_classes(x)

In [18]:
encoder.inverse_transform(predictedLabel)

/Users/pangbo/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([u'Professional', u'Food&Drink', u'Food&Drink', u'Sports',
       u'Professional', u'Sports', u'Food&Drink', u'Food&Drink',
       u'Food&Drink', u'Professional', u'Food&Drink', u'Professional',
       u'Food&Drink', u'Nightlife', u'Sports', u'Food&Drink',
       u'Food&Drink', u'Shows', u'Shows', u'Shows', u'Professional',
       u'Shows', u'Professional', u'Professional', u'Professional',
       u'Art', u'Professional', u'Food&Drink', u'Shows'], dtype='<U12')

In [19]:
predictedLabel

array([6, 4, 4, 8, 6, 8, 4, 4, 4, 6, 4, 6, 4, 5, 8, 4, 4, 7, 7, 7, 6, 7,
       6, 6, 6, 0, 6, 4, 7])